In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import math
import pandas as pd
import pickle
import seaborn as sns

import noise
import random

## Distribuzioni

In [2]:
df_train = pd.read_pickle("data/df_train.pkl")
df_test = pd.read_pickle("data/df_test.pkl")

In [ ]:
len(df_train), len(df_test)

In [ ]:
# ultrasonic distance distribution
def plot_distance_distribution(df, title):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    sns.kdeplot(df['distance'], ax=ax, fill=True)
    plt.title(f'Training Room - Distance distribution of the {title} set')
    ax.set_xlabel('Distance [m]')
    plt.ylim(0, 0.2)
    plt.xlim(-1, 7.5)
    plt.grid()
    plt.show()
    
    # bins representation
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    sns.histplot(df['distance'], ax=ax, bins=100)
    plt.title(f'Training Room - Distance distribution of the {title} set')
    ax.set_xlabel('Distance [m]')
    plt.show()

plot_distance_distribution(df_train, 'training')
plot_distance_distribution(df_test, 'test')

In [ ]:
# pose distribution (x, y, z, yaw)
def plot_distribution(df, title):
    x = df['x']
    y = df['y']
    z = df['z']
    theta = df['theta']

    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))

    # sns.kdeplot(x, ax=axs[0, 0], color='orange', fill=True)
    sns.histplot(x, ax=axs[0, 0], bins=100, color='orange')
    axs[0, 0].set_xlabel('Output: x [m]')
    axs[0, 0].set_xlim(-12, 12)
    axs[0, 0].grid(True)

    sns.kdeplot(y, ax=axs[0, 1], color='red', fill=True)
    sns.histplot(y, ax=axs[0, 1], bins=100, color='red')
    axs[0, 1].set_xlabel('Output: y [m]')
    axs[0, 1].set_xlim(-6, 6)
    axs[0, 1].grid(True)


    # sns.kdeplot(z, ax=axs[1, 0], color='purple', fill=True)
    sns.histplot(z, ax=axs[1, 0], bins=100, color='purple')
    axs[1, 0].set_xlabel('Output: z [m]')
    axs[1, 0].set_xlim(-1, 5)
    axs[1, 0].grid(True)

    # sns.kdeplot(theta, ax=axs[1, 1], color='darkviolet', fill=True)
    sns.histplot(theta, ax=axs[1, 1], bins=100, color='darkviolet')
    axs[1, 1].set_xlabel('Output: θ [rad]')
    axs[1, 1].set_xlim(-1, 2*math.pi + 1)
    axs[1, 1].grid(True)


    fig.suptitle(f'Training room - Sample distribution of {title} set')
    plt.tight_layout()
    plt.show()

plot_distribution(df_train, "training")
plot_distribution(df_test, "test")


## DroneNet (IMAV)

In [ ]:
df_train = pd.read_pickle("data/df_train.pkl")
df_test = pd.read_pickle("data/df_test.pkl")

# if you want new columns for the labels
# df_train["label_20"] = "" or label_160
# df_test["label_20"] = "" or label_160

In [ ]:
# check if at least 10% of the pixels are within 2 meters, if so the portion of the image is considered unsafe (0), otherwise safe (1)
def perform_thresholding(image):
    
    # 255 : 6.4516 = x : 2
    x = (2 * 255) / 6.4516
    
    # pixels within 2 meters
    pixels_within_threshold = np.sum(image <= x)
    
    total_pixels = np.prod(image.shape)
    
    # percentage of pixels within 2 meters
    percentage = pixels_within_threshold / total_pixels * 100
    
    # 0 = unsafe, 1 = safe
    if percentage >= 10:
        return 0
    else:
        return 1

In [ ]:
# define the labels for the images checking if the portion of the image is safe or not
# remember to switch between 20x20 and 160x160 since the models have different output sizes
def write_labels(df):
    
    for idx in df.index:
        depth_img = np.array(df.at[idx, 'depth_img'])
        
        depth_img = depth_img[:, :, 0]
        
        #resize image
        depth_img = Image.fromarray(depth_img)
        depth_img = depth_img.resize((20, 20))
        # depth_img = depth_img.resize((160, 160))
        depth_img = np.array(depth_img)
        
        
        left = depth_img[:, :20 // 3]
        center = depth_img[:, 20 // 3: 2 * 20 // 3]
        right = depth_img[:, 2 * 20 // 3:]
        
        # left = depth_img[:, :160 // 3]
        # center = depth_img[:, 160 // 3: 2 * 160 // 3]
        # right = depth_img[:, 2 * 160 // 3:]
        
        left_prob_collision = perform_thresholding(left)
        center_prob_collision = perform_thresholding(center)
        right_prob_collision = perform_thresholding(right)
        
        
        df.at[idx, 'label_20'] = [left_prob_collision, center_prob_collision, right_prob_collision]
        # df.at[idx, 'label_160'] = [left_prob_collision, center_prob_collision, right_prob_collision]
    
    return df

In [ ]:
df_train = write_labels(df_train)
df_test = write_labels(df_test)

In [ ]:
# save the new dataframes
# df_train.to_pickle("data/df_train.pkl")
# df_test.to_pickle('data/df_test.pkl')

## Perline Noise

In [ ]:
df_train = pd.read_pickle("data/df_train.pkl")
df_test = pd.read_pickle("data/df_test.pkl")

# if you want new columns for the noisy images
# df_train["noisy_image"] = ""
# df_test["noisy_image"] = ""

In [ ]:
# generate perlin noise mask
def generate_perlin_noise(width, height, scale=100):
    noise_array = np.zeros((height, width))
    octave = np.random.randint(1, 10)
    base = random.randint(0, 100)
    for y in range(height):
        for x in range(width):
            noise_array[y][x] = noise.pnoise2(
                x / scale,
                y / scale,
                octaves=octave,
                base=base,
            )
            
    return noise_array

# apply perlin noise to the entire image and return the noisy image
def apply_perlin_noise_entire_image(img, dth, scale=100, train_flag=True):
    width, height = img.shape[0], img.shape[1]

    img_array = np.array(img)
    
    # Generate Perlin noise and normalize it
    noise_array = generate_perlin_noise(width, height, scale)
    noise_array = (noise_array - noise_array.min()) / (noise_array.max() - noise_array.min())
    
    if train_flag:
        alpha = random.uniform(0, 1)
    else:
        alpha = random.uniform(0.3, 1)
    
    interpolation_value = random.uniform(0, 1)
    noise_array = noise_array * alpha
    
    noisy_img_array_1 = img_array * (1 - interpolation_value) + noise_array * interpolation_value
    
    return noisy_img_array_, alpha

    # smoke that depends on the distance given the depth image (tried but not used)
    # noise_array_ = np.multiply(noise_array * alpha, dth)
    # noisy_img_array_ = np.multiply(img_array, (1 - dth)) + noise_array_
    # return noisy_img_array_, alpha

In [ ]:
# add perlin noise image to the dataset
def create_noisy_image(df, train_flag):
    alpha_list = []
    for idx in df.index:
        
        img = df.at[idx, 'img']
        depth = df.at[idx, 'depth_img']
        
        img = img[:, :, 0]
        img = img / 255
        depth = depth[:, :, 0]
        depth = depth / 255
        
        noisy_image, alpha = apply_perlin_noise_entire_image(img, depth, train_flag=train_flag)
        alpha_list.append(alpha)
        
        noisy_image = np.array(noisy_image * 255, dtype=np.uint8)
        
        df.at[idx, 'noisy_image'] = noisy_image
        
        """Qualitative analysis"""
        if random.uniform(0, 1) > 0.97:
            fig, axs = plt.subplots(1, 2, figsize=(8, 5))
            axs[0].imshow(img, cmap='gray')
            axs[0].set_title('Original Image')
            axs[0].xaxis.tick_top()

            axs[1].imshow(depth, cmap='gray')
            axs[1].set_title('Depth Image')
            axs[1].xaxis.tick_top()
            
            axs[2].imshow(noisy_image, cmap='gray')
            axs[2].set_title(f'Noisy Image')
            axs[2].xaxis.tick_top()
            
            plt.tight_layout()
            plt.show()
    
    return df, alpha_list

df_train, alpha_list_train = create_noisy_image(df_train, train=True)
df_test, alpha_list_test = create_noisy_image(df_test, train=False)

In [ ]:
# plot the alpha distribution in a kde plot
def plot_alpha_list(alpha_list, title):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    sns.kdeplot(alpha_list, ax=ax, fill=True)
    plt.title(f'Training Room - Alpha distribution of the {title} set')
    ax.set_xlabel('Alpha')
    plt.ylim(0, 1.5)
    plt.grid()
    plt.show()

# plot the alpha distribution in a histogram
def histo_alpha_list(alpha_list, title):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.hist(alpha_list, bins=30, alpha=0.7, color='b', edgecolor='black')
    plt.title(f'Training Room - Alpha distribution of the {title} set')
    ax.set_xlabel('Alpha')
    plt.show()

plot_alpha_list(alpha_list_train, 'training')
plot_alpha_list(alpha_list_test, 'test')

histo_alpha_list(alpha_list_train, 'training')
histo_alpha_list(alpha_list_test, 'test')

In [ ]:
# save the new dataframes with the noisy images
# df_train.to_pickle("data/df_train.pkl")
# df_test.to_pickle('data/df_test.pkl')